# 04.  TRANSFORMER

1. Seq2seq
2. Transformer
3. Encoder
4. Attention
5. FFN
6. Embeddings and Softmax
7. Positional Encodings
8. Exercise
9. References

# 1. Seq2seq

Seq2seq --- это семейство подходов для задач преобразования последовательностей (например, NLP, ASR, ML4SE).

На текущий момент, самые распространённые подходы --- это подходы на основе Трансформера (энкодер-декодер, декодер или энкодер).

Энкодер-декодерная модель состоит из:
- энкодера (encoder) и
- декодера (decoder).

*Энкодер* отображает входную последовательность представлений символов $(x_1, ..., x_n)$ в последовательность непрерывных представлений $z = (z_1, ..., z_n)$.

*Декодер* отображает $z$ в выходную последовательность $(y_1, ..., y_m)$ символов по одному элементу за раз.

На каждом шаге модель является авторегрессивной, получая ранее сгенерированные символы в качестве дополнительных входных данных при создании следующего.

In [5]:
from IPython.display import IFrame

IFrame(src='http://jalammar.github.io/images/seq2seq_3.mp4', width=800, height=None)

# 2. Transformer

Ранее использовались энкодер-декодерные архитектуры на основе [RNN](https://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html). В 2017 в работе [Vaswani et al - Attention Is All You Need](https://arxiv.org/abs/1706.03762) был предложена архитектура Трансформер (Google Brain).

![paper](res/04_attention_is_all_you_need.png)

Трансформер состоит из энкодера и декодера, каждый из которых состоит из слоёв.

![](https://lena-voita.github.io/resources/lectures/seq2seq/transformer/model-min.png)

#### More
- [Vaswani et al - Attention Is All You Need](https://arxiv.org/abs/1706.03762)
- [Jay Allamar - The Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/)
- [Huang et al - The Annotated Transformer](http://nlp.seas.harvard.edu/annotated-transformer/)
- [Lena Voita - Seq2seq and Attention](https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html)
- [Brandon Rohrer - Transformers from Scratch](https://e2eml.school/transformers.html)
- [Peter Bloem - Transformers from Scratch](https://peterbloem.nl/blog/transformers)

# 3. Encoder

Энкодер состоит из $N$ (например,  $N = 6$) одинаковых последовательных слоёв.
Каждый слой имеет два подслоя:
- multi-head self-attention mechanism, и
- position-wise fully connected feed-forward network.

![](http://jalammar.github.io/images/t/Transformer_encoder.png)

Входная последовательность токенизируется. Каждому токену соответствует вектор (эмбеддинг).
![](http://jalammar.github.io/images/t/encoder_with_tensors_2.png)

Кроме этого используется *residual connection* вокруг каждого из двух подслоев,
за которым следует [layer normalization](https://arxiv.org/abs/1607.06450).

Более формально, выход каждого подслоя --- это `LayerNorm(x + Sublayer(x))`, где `Sublayer(x)` --- это функция, реализованная самим подслоём.

![](https://www.researchgate.net/publication/334288604/figure/fig1/AS:778232232148992@1562556431066/The-Transformer-encoder-structure_W640.jpg)

Все подслои в модели и выходные эмбеддинги имеют одинаковую размерность $d_{model}$ (например, $d_{model} = 512$).

#### More
- [Ulf Mertens - The Transformer encoder](https://github.com/mertensu/transformer-tutorial/blob/master/transformer_encoder.ipynb)

# 4. Self-attention

Attention можно описать как функцию, отображающую *запрос (query)* и множество пар *ключ-значение (key-value)* в некоторый выход.

![](http://jalammar.github.io/images/t/transformer_self_attention_vectors.png)

Запрос, ключи, значения и выход являются векторами.
Вектор вычисляется как взвешенная сумма значений, где вес, присвоенный каждому значению, вычисляется по паре "запрос-ключ".

![](https://i.stack.imgur.com/SG66z.png)

#### Scaled Dot-Product Attention

На входе мы имеем:
- запросы и ключи ключей размерности $d_k$ и
- значений размерности $d_v$.

Далее вычисляем скалярные произведения запроса со всеми ключами, делим каждое такое произведение на $\sqrt{d_k}$.

Затем применяем softmax для получения весов, с которыми будем суммировать значения.

![](res/04_scaled_dot-product_attention.png)

На практике вычисления происходят в матричной форме (одновременно для набора запросов, упакованных вместе в матрицу $Q$). Ключи и значения также упаковываются вместе в матрицы $K$ и $V$.

Таким образом,
$$Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V$$

Деление на $\sqrt{d_k}$ позволяет уменьшить значение больших скалярных произведений.
(Если они большие, то функция softmax в таком случае имеет очень маленькие градиенты. Это плохо.)

![](https://jalammar.github.io/images/t/self-attention-output.png)

В матричной записи:

![](http://jalammar.github.io/images/t/self-attention-matrix-calculation-2.png)

#### Multi-Head Attention

Multi-Head Attention позволяет запустить несколько attention-механизмов, каждый из которых может обладать своими полезными особенностями.
Таким образом собирается информация из различных представлений.

![](https://i.ytimg.com/vi/mMa2PmYJlCo/mqdefault.jpg)

Поэтому с помощью линейных преобразований (projections) получаем несколько ($h$) троек запрос-ключ-значение (размерности, соответственно, $d_k$, $d_k$, $d_v$).

Затем для каждой из этих троек параллельно запускается attention-механизм, что даёт выходные значения размерности $d_v$.
Далее полученные выходные значения конкатенируются и снова проецируются.

![](https://repository-images.githubusercontent.com/283979760/0f00ed80-d368-11ea-979d-78033d0a1cee)

Более формально:

$$MultiHead(Q, K, V ) = concat(head_1, ..., head_h)W^O$$

где $head_i = Attention(QW^Q_i, KW^K_i, V W^V_i)$ и используются следующие матрицы проецирования

$W^Q_i \in R^{d_{model} \times d_k}$,

$W^K_i \in R^{d_{model} \times d_k}$,

$W^V_i \in R^{d_{model} \times d_v}$,

$W^O_i \in R^{hd_v \times d_{model}}$.

Размерности могут быть, например, следующими $h = 8$, $d_k = d_v = \frac{d_{model}}{h} = 64$.

Трансформер использует multi-head attention тремя различными способами:

1. В слоях "энкодер-декодер attention" запросы поступают с предыдущего уровня декодера, а ключи и значения поступают с выходных данных энкодера. Это позволяет каждой позиции в декодере посетить все позиции входной последовательности.

2. Энкодер содержит self-attention слои.
Здесь все ключи, значения и запросы поступают из одного и того же места --- из вывода предыдущего слоя энкодера. Каждая позиция в энкодере может посетить все позиции предыдущего слоя энкодера.

3. Декодер содержит self-attention слои.
Здесь допускается для каждой позиции декодера посетить все предыдущие позиции (слева) до текущей позии включительно.
Такое ограничение возникает, поскольку необходимо сохранить авторегрессионное свойство декодера (по предыдущим токенам предсказывать следующий).
Ограничение реализуются внутри scaled dot-product attention максированием (полагая $-\infty$) всех значений на входе softmax-а, которые соответствуют недопустимым посещениям.

# 5. FFN

Каждый из слоёв энкодера и декодера содержит FFN. Этот подслой одинаково применяется к каждой позиции отдельно.

![](https://jalammar.github.io/images/t/encoder_with_tensors_2.png)
Он состоит из двух линейных преобразований с активацией [ReLU](https://arxiv.org/abs/1803.08375) между ними.

$$FFN(x) = \max(0, xW_1 + b_1)W_2 + b_2$$

Хотя линейные преобразования одинаковы для разных позиций,
  они используют разные параметры от слоя к слою.

Внутренний слой имеет большую размерностью.
Например, размерность входа и выхода $d_{model} = 512$, размерность внутреннего слоя $d_{ff} = 2048$.

В частности мы имеем, что в Трансформере токен в каждой позиции проходит свой собственный путь в энкодере.
Есть зависимости на уровне self-attention слоёв, но нет зависимостей на уровне FFN.
Это позволяет проходить FFN-подслои параллельно.

# 6. Embeddings and softmax

Для преобразования входных/выходных токенов в вектора (размерности $d_{model}$) используются обучаемые эмбеддинги.
Для преобразования выходов декодеров в предсказанные вероятности следующего токена используется обучаемое линейное преобразование и softmax.

Чуть более подробно.  На выходе декодера мы получаем вектор.
Этот вектор проходит через линейный слой (fully connected neural network), за которым следует softmax.

Линейный слой проецирует выход декодера в гораздо больший вектор (logits-вектор).
Размер этого вектор --- это размер словаря токенов.

Затем слой softmax превращает координаты в вероятности.
Выбирается наиболее вероятный токен.


![](https://jalammar.github.io/images/t/transformer_decoder_output_softmax.png)

# 7. Positional encodings

Для того, чтобы использовать информацию о порядке токенов в последовательности, в Трансформере применяются positional encoding. Positional encoding --- это вектор, в котором закодирован номер.
Он имеет ту же длину, что и эмбеддинги ($d_{model}$). И  прибавляется к эмбеддингу на входе.

Например, можно использовать такие функции:
$$PE(pos, 2i) = \sin(\frac{pos}{10000^{\frac{2i}{d_{model}}}})$$
$$PE(pos, 2i+1) = \cos(\frac{pos}{10000^{\frac{2i}{d_{model}}}})$$

где $pos$ --- это индекс позиции, а $i$  --- индекс координаты.

![](https://jalammar.github.io/images/t/attention-is-all-you-need-positional-encoding.png)

Другие варианты:
- обучаемые
- [относительные](https://arxiv.org/abs/1803.02155)
- [RoPE](https://arxiv.org/abs/2104.09864)
  
#### More
- [О методах позиционного кодирования в Transformer](https://habr.com/ru/articles/780116/)
- [Lilian Weng - The Transformer Family](https://lilianweng.github.io/posts/2023-01-27-the-transformer-family-v2/)
- [LongRoPE](https://arxiv.org/abs/2402.13753)

# 8. Batches and Paddings

![](https://i.stack.imgur.com/UOo7qm.png)

# 8. Exercise

Реализовать Encoder с нуля на PyTorch. Быть готовыми ответить на вопросы по коду.

# 9. References

- [Lilian Weng - The Transformer Family](https://lilianweng.github.io/posts/2023-01-27-the-transformer-family-v2/)
- [nn.labml.ai - Paper Implementations](https://nn.labml.ai/)
- [Функция активации ReLU - SwiGLU](https://arxiv.org/abs/2002.05202)
- [Post LayerNorm - Pre LayerNorm](https://arxiv.org/abs/2002.04745)
- [LayerNorm - RMSNorm](https://arxiv.org/abs/1910.07467)
- [Модификации Attention](https://arxiv.org/abs/2305.13245)
- [Andrej Karpathy - minGPT](https://github.com/karpathy/minGPT)
- [pytorch - NLP from Scratch](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)